# AMD 显卡驱动

In [ ]:
%%bash

# 更新系统
echo "正在更新系统..."
sudo pacman -Syu --noconfirm
# 安装必要的开发工具
echo "安装开发工具..."
sudo pacman -Sy --needed --noconfirm base-devel git wget curl cmake

# 安装ROCm驱动和开发工具包
echo "安装ROCm驱动..."
sudo pacman -Sy --needed --noconfirm rocm-hip-sdk rocm-opencl-sdk rocm-core

# 添加用户到必要的硬件访问组
echo "配置用户组权限..."
sudo usermod -a -G render,video,kvm $USER

# 验证安装
echo "验证ROCm安装..."
/opt/rocm/bin/rocminfo


# 将 ROCm 的 bin 目录添加到 PATH 环境变量（如果尚未添加）
echo "将 ROCm 的 bin 目录添加到 PATH 环境变量（如果尚未添加）"
grep -q ':/opt/rocm/bin' <<< $PATH || echo 'export PATH=$PATH:/opt/rocm/bin:/opt/rocm/opencl/bin' >> ~/.zshrc
grep -q ':/opt/rocm/lib' <<< $LD_LIBRARY_PATH || echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/opt/rocm/lib:/opt/rocm/lib64' >> ~/.zshrc

echo "安装完成！请重启系统使所有配置生效。"
echo "重启后使用以下命令激活环境:"
echo "source ~/rocm-env/bin/activate"



# virtualbox


In [ ]:
%%bash

uname -r
sudo pacman -Sy virtualbox linux612-virtualbox-host-modules virtualbox-ext-vnc --noconfirm --needed

# 加载内核模块
sudo modprobe vboxdrv
sudo modprobe vboxnetadp
sudo modprobe vboxnetflt

# 添加用户到 vboxusers 组
sudo usermod -aG vboxusers $USER

newgrp vboxusers

# 重启生效
# reboot


#  Davinci Resolve

In [ ]:
# 1. 访问官网下载页面
# 打开浏览器访问: https://www.blackmagicdesign.com/products/davinciresolve/

# 2. 手动下载Linux版本
# 下载完成后，文件通常在 ~/Downloads/ 目录

# 运行.run文件


# aria2 webui
https://github.com/mayswind/AriaNg/releases

In [ ]:
import os
from pathlib import Path

def setup_aria2_config():
    """Setup aria2 configuration directory and files"""
    
    # Create config directory
    config_dir = Path.home() / ".config" / "aria2"
    config_dir.mkdir(parents=True, exist_ok=True)
    print(f"📁 Created config directory: {config_dir}")
    
    # Create session file
    session_file = config_dir / "aria2.session"
    session_file.touch()
    print(f"📄 Created session file: {session_file}")
    
    # Create configuration file
    config_content = """# 基本设置
dir=${HOME}/Downloads
continue=true
max-connection-per-server=16
min-split-size=1M
split=16

# 进度保存
input-file=${HOME}/.config/aria2/aria2.session
save-session=${HOME}/.config/aria2/aria2.session
save-session-interval=60

# RPC 设置 (用于 WebUI)
enable-rpc=true
rpc-listen-all=true
rpc-listen-port=6800
rpc-secret=mysecret

# BT 设置
bt-enable-lpd=true
bt-max-peers=50
bt-request-peer-speed-limit=1M
bt-stop-timeout=0

# 性能优化
max-overall-download-limit=0
max-overall-upload-limit=1K
seed-ratio=1.0
seed-time=0

# 日志
log=${HOME}/.config/aria2/aria2.log
log-level=warn
"""
    
    config_file = config_dir / "aria2.conf"
    with open(config_file, 'w', encoding='utf-8') as f:
        f.write(config_content)
    print(f"⚙️  Created config file: {config_file}")
    
    print("✅ aria2 configuration completed")

# Run the function
setup_aria2_config()

In [ ]:
import os
from pathlib import Path
import subprocess

def setup_aria2_service():
    """Setup aria2 as a systemd user service"""
    
    # Create systemd user directory
    service_dir = Path.home() / ".config" / "systemd" / "user"
    service_dir.mkdir(parents=True, exist_ok=True)
    print(f"📁 Created service directory: {service_dir}")
    
    # Create service file content
    service_content = """[Unit]
Description=Aria2c Download Manager
After=network.target

[Service]
Type=forking
ExecStart=/usr/bin/aria2c --conf-path=%h/.config/aria2/aria2.conf -D
ExecStop=/usr/bin/killall aria2c
Restart=on-failure

[Install]
WantedBy=default.target
"""
    
    # Write service file
    service_file = service_dir / "aria2.service"
    with open(service_file, 'w', encoding='utf-8') as f:
        f.write(service_content)
    print(f"⚙️  Created service file: {service_file}")
    
    # Reload systemd and enable service
    try:
        subprocess.run(["systemctl", "--user", "daemon-reload"], check=True)
        print("✅ Systemd daemon reloaded")
        
        subprocess.run(["systemctl", "--user", "enable", "aria2"], check=True)
        print("✅ Service enabled")
        
        subprocess.run(["systemctl", "--user", "start", "aria2"], check=True)
        print("✅ Service started")
        
        print("🎉 aria2 system service installed successfully")
        
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to setup service: {e}")
        return False
    
    return True

# Run the function
setup_aria2_service()

In [ ]:
%%bash
# 不使用 jq，通过 grep 和 sed 提取版本号
LATEST_TAG=$(curl -s "https://api.github.com/repos/mayswind/AriaNg/releases/latest" | grep '"tag_name":' | sed -E 's/.*"([^"]+)".*/\1/')
echo "最新版本: $LATEST_TAG"

wget "https://github.com/mayswind/AriaNg/releases/download/${LATEST_TAG}/AriaNg-${LATEST_TAG}-AllInOne.zip"

echo "✅ 下载完成"

# unzip aria2 webui
unzip -o AriaNg-${LATEST_TAG}-AllInOne.zip -d ./bin/AriaNg

rm -f "AriaNg-${LATEST_TAG}-AllInOne.zip"

python3 -m http.server 8080 --directory ./bin/AriaNg


- 打开浏览器访问 http://localhost:8080/
- AriaNg 设置
- RPC
- Aria2 RPC 协议
- websocket
- 密钥

# ollama


In [ ]:

curl -fsSL https://ollama.com/install.sh | sh

# 启动Ollama服务
sudo systemctl start ollama
# 关闭Ollama服务
# 检查服务状态
sudo systemctl status ollama
# 更改模型位置
sudo systemctl edit ollama.service
[Service]
Environment="OLLAMA_MODELS=/home/kf/.ollama/models"

sudo systemctl daemon-reload
sudo systemctl restart ollama
# 检查GPU内存
rocm-smi


# shadowsocks

In [ ]:
%%bash
#!bash
# 服务器端manjaro版本
sudo pacman -Sy shadowsocks --noconfirm --needed
# 设置配置变量
SERVER="0.0.0.0"
PORT=8388
PASSWORD="vpntest"
METHOD="aes-256-gcm"
TIMEOUT=300

# 创建配置文件
sudo mkdir -p /etc/shadowsocks
sudo tee /etc/shadowsocks/config.json > /dev/null << EOF
{
    "server": "$SERVER",
    "server_port": $PORT,
    "password": "$PASSWORD",
    "method": "$METHOD",
    "timeout": $TIMEOUT,
    "fast_open": false,
    "mode": "tcp_and_udp"
}
EOF

# 验证配置
sudo cat /etc/shadowsocks/config.json
sudo systemctl start shadowsocks-server@config
sudo systemctl status shadowsocks-server@config

# 添加 iptables 规则开放8388端口
sudo iptables -A INPUT -p tcp --dport 8388 -j ACCEPT
sudo iptables -A INPUT -p udp --dport 8388 -j ACCEPT

# 保存 iptables 规则（根据系统选择合适的方法）
sudo iptables-save | sudo tee /etc/iptables/iptables.rules

# 检查规则是否添加成功
sudo iptables -L INPUT -n | grep 8388

# 成功


In [ ]:
%%bash
# 客户端
sudo pacman -Sy shadowsocks --noconfirm --needed
SERVER="207.148.77.242"
PASSWORD="vpntest"
LOCAL_PORT=1080

sudo tee /etc/shadowsocks/config.json > /dev/null << EOF
{
    "server": "$SERVER",
    "server_port": 8388,
    "local_address": "127.0.0.1",
    "local_port": "$LOCAL_PORT",
    "password": "$PASSWORD",
    "method": "aes-256-gcm",
    "timeout": 300
}
EOF
cat /etc/shadowsocks/config.json
sudo systemctl start shadowsocks@config 
sudo systemctl status shadowsocks@config   
sudo systemctl stop shadowsocks@config  

curl --socks5 127.0.0.1:1080 ipinfo.io/ip

In [ ]:
# shadowsocks

# 服务器端 ubuntu版本
sudo apt update
sudo apt install shadowsocks-libev
sudo nano /etc/shadowsocks-libev/config.json
{
    "server": "0.0.0.0",
    "server_port": 8388,
    "password": "your_password_here",
    "method": "aes-256-gcm",
    "timeout": 300,
    "fast_open": false
}

# 启动 Shadowsocks 服务
sudo systemctl start shadowsocks-libev

# 设置开机自动启动
sudo systemctl enable shadowsocks-libev

# 检查服务状态，确认是否正常运行
sudo systemctl status shadowsocks-libev
# 开放 Shadowsocks 服务端口
sudo ufw allow 8388/tcp

# 重新加载防火墙规则
sudo ufw reload

# 检查防火墙状态，确认端口已开放
sudo ufw status

# 检查8388端口是否在监听
netstat -tlnp | grep 8388

# 或者使用 ss 命令
ss -tlnp | grep 8388


In [ ]:
%%bash
# ssh: connect to host 207.148.77.242 port 22: Connection refused
ssh-keygen -R 207.148.77.242 

# clash-verge

In [ ]:

import yaml
from pathlib import Path

def create_clash_config_cn(config_path="~/.config/clash/config.yaml"):
    """
    创建中文注释的 Clash 配置文件
    """
    config = {
        # HTTP(S) 代理服务端口
        'port': 7890,
        
        # SOCKS5 代理服务端口  
        'socks-port': 7891,
        
        # 设置为 true 以允许来自其他 LAN IP 地址的连接
        'allow-lan': False,
        
        # Clash 路由工作模式
        # rule: 基于规则的数据包路由
        # global: 所有数据包将被转发到单个节点  
        # direct: 直接将数据包转发到互联网
        'mode': 'rule',
        
        # 日志级别: info / warning / error / debug / silent
        'log-level': 'info',
        
        # RESTful Web API 监听地址
        'external-controller': '127.0.0.1:9090',
        
        # 代理服务器配置
        'proxies': [],
        
        # 代理组配置
        'proxy-groups': [
            {
                'name': '🚀 代理',
                'type': 'select',
                'proxies': ['DIRECT']
            }
        ],
        
        # 规则配置
        'rules': [
            # 本地网络直连
            'DOMAIN-SUFFIX,local,DIRECT',
            'IP-CIDR,127.0.0.0/8,DIRECT', 
            'IP-CIDR,10.0.0.0/8,DIRECT',
            'IP-CIDR,172.16.0.0/12,DIRECT',
            'IP-CIDR,192.168.0.0/16,DIRECT',
            
            # 国内网站直连
            'GEOIP,CN,DIRECT',
            
            # 常用国外网站走代理
            'DOMAIN-SUFFIX,google.com,🚀 代理',
            'DOMAIN-SUFFIX,gstatic.com,🚀 代理', 
            'DOMAIN-SUFFIX,googleapis.com,🚀 代理',
            'DOMAIN-SUFFIX,github.com,🚀 代理',
            'DOMAIN-SUFFIX,github.io,🚀 代理',
            'DOMAIN-SUFFIX,githubassets.com,🚀 代理',
            'DOMAIN-SUFFIX,gitlab.com,🚀 代理',
            'DOMAIN-SUFFIX,youtube.com,🚀 代理',
            'DOMAIN-SUFFIX,ytimg.com,🚀 代理',
            'DOMAIN-SUFFIX,twitter.com,🚀 代理',
            'DOMAIN-SUFFIX,facebook.com,🚀 代理',
            'DOMAIN-SUFFIX,instagram.com,🚀 代理',
            'DOMAIN-SUFFIX,whatsapp.com,🚀 代理',
            'DOMAIN-SUFFIX,telegram.org,🚀 代理',
            'DOMAIN-SUFFIX,reddit.com,🚀 代理',
            'DOMAIN-SUFFIX,wikipedia.org,🚀 代理',
            'DOMAIN-SUFFIX,stackoverflow.com,🚀 代理',
            'DOMAIN-SUFFIX,stackexchange.com,🚀 代理',
            'DOMAIN-SUFFIX,medium.com,🚀 代理',
            'DOMAIN-SUFFIX,dropbox.com,🚀 代理',
            'DOMAIN-SUFFIX,notion.so,🚀 代理',
            'DOMAIN-SUFFIX,figma.com,🚀 代理',
            'DOMAIN-SUFFIX,discord.com,🚀 代理',
            'DOMAIN-SUFFIX,slack.com,🚀 代理',
            'DOMAIN-SUFFIX,zoom.us,🚀 代理',
            
            # 广告屏蔽
            'DOMAIN-KEYWORD,adservice,REJECT',
            'DOMAIN-SUFFIX,doubleclick.net,REJECT',
            'DOMAIN-SUFFIX,googlesyndication.com,REJECT',
            
            # 最终规则
            'MATCH,🚀 代理'
        ]
    }
    
    config_path = Path(config_path).expanduser()
    config_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(config_path, 'w', encoding='utf-8') as f:
        yaml.dump(config, f, allow_unicode=True, default_flow_style=False, sort_keys=False)
    
    print(f"✅ Clash 配置文件已创建: {config_path}")
    print("📝 现在你可以在 'proxies' 部分添加你的代理服务器")
    print("📍 HTTP 代理: 127.0.0.1:7890")
    print("📍 SOCKS5 代理: 127.0.0.1:7891")
    print("🌐 控制面板: http://127.0.0.1:9090")
    
    return config_path

# 创建配置文件
create_clash_config_cn()

In [ ]:
def add_proxies_to_config(config_path, proxies):
    """
    Add proxies to existing Clash config file
    
    Args:
        config_path: Path to config file
        proxies: List of proxy configurations
    """
    config_path = Path(config_path).expanduser()
    
    if not config_path.exists():
        print("❌ Config file not found, creating new one...")
        create_clash_config_cn(config_path)
    
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    
    # Add new proxies
    config['proxies'].extend(proxies)
    
    # Update proxy groups with new proxies
    proxy_names = [p['name'] for p in proxies if p['type'] not in ['direct', 'reject']]
    
    for group in config['proxy-groups']:
        if group['name'] not in ['DIRECT', 'REJECT']:
            # Add new proxies to existing groups (avoid duplicates)
            for proxy_name in proxy_names:
                if proxy_name not in group['proxies']:
                    group['proxies'].append(proxy_name)
    
    with open(config_path, 'w', encoding='utf-8') as f:
        yaml.dump(config, f, allow_unicode=True, default_flow_style=False, sort_keys=False)
    
    print(f"✅ Added {len(proxies)} proxies to config")
    return config_path

# Example usage to add proxies later:
"""
# Define your proxies
my_proxies = [
    {
        'name': 'My-SS-Server',
        'type': 'ss',
        'server': 'example.com',
        'port': 8388,
        'cipher': 'chacha20-ietf-poly1305',
        'password': 'mypassword',
        'udp': True
    }
]

# Add them to config
add_proxies_to_config('~/.config/clash/config.yaml', my_proxies)
"""
myproxies=my_proxies = [
    {
        'name': 'My-SS-Server',
        'type': 'ss',
        'server': 'example.com',
        'port': 8388,
        'cipher': 'chacha20-ietf-poly1305',
        'password': 'mypassword',
        'udp': True
    }
]
# Add them to config
add_proxies_to_config('~/.config/clash/config.yaml', my_proxies)

In [ ]:
# 终端启动代理
export ALL_PROXY="socks5://192.168.0.101:7897"

In [ ]:
# 终端代理
!google-chrome-stable  --proxy-server="socks5://192.168.0.101:7897" 

# google chrome



In [ ]:
# 使用代理启动谷歌浏览器
!google-chrome-stable  --proxy-server="socks5://192.168.0.101:7897"   

## floccus Sync bookmarks

## Proxy SwitchyOmega 3


# yabridge

In [ ]:


```bash
sudo pacman -Sy --needed --noconfirm yabridge yabridgectl wine wine-gecko wine-mono  winetricks 

# Acon Digital
WINEPREFIX=/run/media/kf/WD/wine/vst/AconDigital winecfg
yabridgectl add /run/media/kf/WD/wine/vst/AconDigital/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# Antares
No success
WINEPREFIX=/run/media/kf/WD/wine/vst/AntaresAutoTune winetricks dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/AntaresAutoTune wine Antares\ Auto-Tune\ Pro\ v11.0.0.exe 
yabridgectl add /run/media/kf/WD/wine/vst/AntaresAutoTune/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/AntaresDeEsser wine 
yabridgectl add /run/media/kf/WD/wine/vst/AntaresDeEsser/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# Arturia
WINEPREFIX=/run/media/kf/WD/wine/vst/Arturia winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/Arturia wine 
yabridgectl add /run/media/kf/WD/wine/vst/Arturia/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# BABY Audio
WINEPREFIX=/run/media/kf/WD/wine/vst/BABYAudio winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/BABYAudio winecfg
yabridgectl add /run/media/kf/WD/wine/vst/BABYAudio/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

#BBE  
WINEPREFIX=/run/media/kf/WD/wine/vst/BBESonicSweet winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/BBESonicSweet wine s
yabridgectl add /run/media/kf/WD/wine/vst/BBESonicSweet/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/BBEStompBoard  winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/BBEStompBoard wine s
yabridgectl add /run/media/kf/WD/wine/vst/BBEStompBoard/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# Boz Digital Labs
WINEPREFIX=/run/media/kf/WD/wine/vst/BozDigitalLabs wine 
yabridgectl add /run/media/kf/WD/wine/vst/BozDigitalLabs/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# CableGuys
WINEPREFIX=/run/media/kf/WD/wine/vst/CableGuysShaperBox winetricks dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/CableGuysShaperBox wine 
yabridgectl add /run/media/kf/WD/wine/vst/CableGuysShaperBox/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/CableGuysKickStart winetricks dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/CableGuysKickStart wine 
yabridgectl add /run/media/kf/WD/wine/vst/CableGuysKickStart/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# CamelCrusher
WINEPREFIX=/run/media/kf/WD/wine/vst/CamelCrusher wine 
yabridgectl add /run/media/kf/WD/wine/vst/CamelCrusher/drive_c/Program\ Files/VSTPlugins/
yabridgectl sync --prune

# Celemony
WINEPREFIX=/run/media/kf/WD/wine/vst/Celemony winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/Celemony wine 
yabridgectl add /run/media/kf/WD/wine/vst/Celemony/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# cradle
WINEPREFIX=/run/media/kf/WD/wine/vst/Cradle winetricks dxvk 
WINEPREFIX=/run/media/kf/WD/wine/vst/Cradle wine 
yabridgectl add /run/media/kf/WD/wine/vst/Cradle/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# d16grouprepeater
WINEPREFIX=/run/media/kf/WD/wine/vst/D16GroupRepeater winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/D16GroupRepeater wine s
yabridgectl add /run/media/kf/WD/wine/vst/D16GroupRepeater/drive_c/Program\ Files//Steinberg/VSTPlugins/
yabridgectl sync --prune

#dadalife
no success

# deniseaudio
WINEPREFIX=/run/media/kf/WD/wine/vst/DeniseAudio winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/DeniseAudio wine 
yabridgectl add /run/media/kf/WD/wine/vst/DeniseAudio/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# exciteaudio
WINEPREFIX=/run/media/kf/WD/wine/vst/ExciteAudioHarmonic winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/ExciteAudioHarmonic wine s
yabridgectl add /run/media/kf/WD/wine/vst/ExciteAudioHarmonic/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune


# fabfilter
WINEPREFIX=/run/media/kf/WD/wine/vst/FabFilter winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/FabFilter wine s
yabridgectl add /run/media/kf/WD/wine/vst/FabFilter/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# goodhertz
WINEPREFIX=/run/media/kf/WD/wine/vst/Goodhertz winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/Goodhertz wine 
yabridgectl add /run/media/kf/WD/wine/vst/Goodhertz/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune
WINEPREFIX=/run/media/kf/WD/wine/vst/Goodhertz wine cmd /c 
WINEPREFIX=/run/media/kf/WD/wine/vst/Goodhertz wine regedit 

# Harman Audio
WINEPREFIX=/run/media/kf/WD/wine/vst/LexiconLXP winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/LexiconLXP wine 
yabridgectl add /run/media/kf/WD/wine/vst/LexiconLXP/drive_c/Program\ Files/Steinberg/VSTPlugins/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/LexiconPCM winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/LexiconPCM wine 
yabridgectl add /run/media/kf/WD/wine/vst/LexiconPCM/drive_c/Program\ Files/Steinberg/VSTPlugins/
yabridgectl sync --prune

# Heckmann Audio - u-he Diva v1.4.8
WINEPREFIX=/run/media/kf/WD/wine/vst/uheDiva wine 
yabridgectl add /run/media/kf/WD/wine/vst/uheDiva/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# ikmultimedia
WINEPREFIX=/run/media/kf/WD/wine/vst/IKMultimediaTracks6 winetricks mfc42 dxvk  
WINEPREFIX=/run/media/kf/WD/wine/vst/IKMultimediaTracks6 wine 
yabridgectl add /run/media/kf/WD/wine/vst/IKMultimediaTracks6/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# izotope
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeAurora winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeAurora wine 
yabridgectl add /run/media/kf/WD/wine/vst/IzotopeAurora/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeRX11 winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeRX11 wine 
yabridgectl add /run/media/kf/WD/wine/vst/IzotopeRX11/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune
###############
# black screen
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeOzone11 winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeOzone11 wine 
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeOnectar winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/IzotopeOnectar wine
###############

# MinimalAudio
WINEPREFIX=/run/media/kf/WD/wine/vst/MinimalAudio winetricks  dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/MinimalAudio wine 
yabridgectl add /run/media/kf/WD/wine/vst/MinimalAudio/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# Native Instruments
WINEPREFIX=/run/media/kf/WD/wine/vst/NativeInstruments winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/NativeInstruments wine *.exe
yabridgectl add /run/media/kf/WD/wine/vst/NativeInstruments/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# NembriniAudio

WINEPREFIX=/run/media/kf/WD/wine/vst/NembriniAudio winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/NembriniAudio wine 
yabridgectl add /run/media/kf/WD/wine/vst/NembriniAudio/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# okesound
WINEPREFIX=/run/media/kf/WD/wine/vst/Okesound winecfg
yabridgectl add /run/media/kf/WD/wine/vst/Okesound/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# overloud 
WINEPREFIX=/run/media/kf/WD/wine/vst/Overloud winetricks dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/Overloud wine 
yabridgectl add /run/media/kf/WD/wine/vst/Overloud/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# pluginalliance
WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceBPE winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceBPE wine s
yabridgectl add /run/media/kf/WD/wine/vst/PluginAllianceBPE/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceFAS wine 
yabridgectl add /run/media/kf/WD/wine/vst/PluginAllianceFAS/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceTTS winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceTTS wine 
yabridgectl add /run/media/kf/WD/wine/vst/PluginAllianceTTS/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/PluginAllianceCopy winecfg
yabridgectl add /run/media/kf/WD/wine/vst/PluginAllianceCopy/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# PolyverseWider
WINEPREFIX=/run/media/kf/WD/wine/vst/PolyverseWider winetricks dxbk
WINEPREFIX=/run/media/kf/WD/wine/vst/PolyverseWider wine 
yabridgectl add /run/media/kf/WD/wine/vst/PolyverseWider/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# pulsar
no success
# RDGA
WINEPREFIX=/run/media/kf/WD/wine/vst/RDGA wine 
yabridgectl add /run/media/kf/WD/wine/vst/RDGA/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# slatedigital
WINEPREFIX=/run/media/kf/WD/wine/vst/Slatedigital winecfg
yabridgectl add /run/media/kf/WD/wine/vst/Slatedigital/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/Slatedigitalana2 winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/Slatedigitalana2 wine 
yabridgectl add /run/media/kf/WD/wine/vst/Slatedigitalana2/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# softube
no success

# ssl
WINEPREFIX=/run/media/kf/WD/wine/vst/SSL winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/SSL winecfg
yabridgectl add /run/media/kf/WD/wine/vst/SSL/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# soundtoys
no success

# sonible
WINEPREFIX=/run/media/kf/WD/wine/vst/SonibleDeEss winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/SonibleDeEss wine 
yabridgectl add /run/media/kf/WD/wine/vst/SonibleDeEss/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/SonibleSmartEQ4 winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/SonibleSmartEQ4 wine 
yabridgectl add /run/media/kf/WD/wine/vst/SonibleSmartEQ4/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/Sonible winetricks mfc42
WINEPREFIX=/run/media/kf/WD/wine/vst/Sonible wine 
yabridgectl add /run/media/kf/WD/wine/vst/Sonible/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# Soundtoys
WINEPREFIX=/run/media/kf/WD/wine/vst/Soundtoys wine 
yabridgectl add /run/media/kf/WD/wine/vst/Soundtoys/drive_c/Program\ Files\ \(x86\)/VstPlugins/
yabridgectl sync --prune

# SpectrasonicsOmnishpere
WINEPREFIX=/run/media/kf/WD/wine/vst/SpectrasonicsOmnisphere winetricks  dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/SpectrasonicsOmnisphere wine 
yabridgectl add /run/media/kf/WD/wine/vst/SpectrasonicsOmnisphere/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# TDeEsser
WINEPREFIX=/run/media/kf/WD/wine/vst/TDeEsser wine 
yabridgectl add /run/media/kf/WD/wine/vst/TDeEsser/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# unitedplugin
WINEPREFIX=/run/media/kf/WD/wine/vst/UnitedPluginFS wine 
yabridgectl add /run/media/kf/WD/wine/vst/UnitedPluginFS/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/UnitedPluginIA wine
yabridgectl add /run/media/kf/WD/wine/vst/UnitedPluginIA/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/UnitedPluginJMGS wine
yabridgectl add /run/media/kf/WD/wine/vst/UnitedPluginJMGS/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/UnitedPluginMA wine
yabridgectl add /run/media/kf/WD/wine/vst/UnitedPluginMA/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

WINEPREFIX=/run/media/kf/WD/wine/vst/UnitedPluginSD wine
yabridgectl add /run/media/kf/WD/wine/vst/UnitedPluginSD/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# ValhallaDSP
WINEPREFIX=/run/media/kf/WD/wine/vst/ValhallaDSP wine 
yabridgectl add /run/media/kf/WD/wine/vst/ValhallaDSP/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# voxengo
WINEPREFIX=/run/media/kf/WD/wine/vst/Voxengo winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/Voxengo wine 
yabridgectl add /run/media/kf/WD/wine/vst/Voxengo/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# waves
no success

# WAKSHMRReverb
WINEPREFIX=/run/media/kf/WD/wine/vst/WAKSHMRReverb wine 
yabridgectl add /run/media/kf/WD/wine/vst/WAKSHMRReverb/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# wavesfactory
WINEPREFIX=/run/media/kf/WD/wine/vst/WavesFactory winetricks mfc42 dxvk
WINEPREFIX=/run/media/kf/WD/wine/vst/WavesFactory wine 
yabridgectl add /run/media/kf/WD/wine/vst/WavesFactory/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# XLNRC20
WINEPREFIX=/run/media/kf/WD/wine/vst/XLNRC20 winetricks mfc42 
WINEPREFIX=/run/media/kf/WD/wine/vst/XLNRC20 wine 
yabridgectl add /run/media/kf/WD/wine/vst/XLNRC20/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# youlean 
WINEPREFIX=/run/media/kf/WD/wine/vst/Youlean wine 
yabridgectl add /run/media/kf/WD/wine/vst/Youlean/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# zynaptiaa
No success
WINEPREFIX=/run/media/kf/WD/wine/vst/Zynaptia winetricks mfc42 vcrun2022
WINEPREFIX=/run/media/kf/WD/wine/vst/Zynaptia wine 
yabridgectl add /run/media/kf/WD/wine/vst/Zynaptia/drive_c/Program\ Files/Common\ Files/VST3/
yabridgectl sync --prune

# 添加 vst3 插件
yabridgectl add path/to/vst3/plugin
yabridgec tl sync --prune

```

## wine


In [ ]:

# GTK+工具
sudo pacman -Sy --needed --noconfirm zenity  
sudo pacman -Sy --needed --noconfirm gnutls lib32-gnutls lib32-alsa-lib lib32-pulseaudio

# xstudio
WINEPREFIX=/run/media/kf/sdb/Linux/exe/XStudio winetricks dxvk cjkfonts vcrun2022
WINEPREFIX=/run/media/kf/sdb/Linux/exe/XStudio wine  
# wechat
WINEPREFIX=/run/media/kf/sdb/Linux/exe/WeChat winetricks dxvk vcrun2022  cjkfonts
WINEPREFIX=/run/media/kf/sdb/Linux/exe/WeChat wine 



# 常见问题处理


In [ ]:
%%bash
# 安装被锁定的问题

sudo rm /var/lib/pacman/db.lck

# 无法注册数据库

sudo mv /etc/lsb-release /etc/lsb-release.bak

sudo rm -rf /etc/lsb-release.bak

# 无法提交处理（有冲突的文件）

# 编辑 /etc/pacman.conf 文件，修改服务器地址

sudo gedit /etc/pacman.conf

# 更新失败

sudo pacman -Syu --ignore <package_name>


# 邮件


In [ ]:

19157521820@163.com

18285998868@163.com

3124568493@qq.com

kefu1820@gmail.com


# Account

- +86 19157521820
- google chrome kefu1820@gmail.com